In [1]:
import numpy as np

In [2]:
class QuadraticProblem:
    # (pseudo) private method
    @staticmethod
    def __check_shape(x, dim=None, varname=""):
        if isinstance(x, np.ndarray):
            if dim == None or x.shape == dim:
                return x
            else: raise TypeError(f"{varname} has shape {x.shape} expected {dim}")
        else: raise TypeError(f"{varname} is not {type({np.ndarray})}")
        
    def __init__(self, A, b, c, H, M, q=None, r=None, tol=1e-8):
        self.tol = tol
            
        # shape of A is assumed to be correct
        self.A = self.__check_shape(A, varname="A")
        m, n = A.shape
        
        self.b = self.__check_shape(b, dim=(m,), varname="b")
        self.c = self.__check_shape(c, dim=(n,), varname="c")
        self.H = self.__check_shape(H, dim=(n,n), varname="H")
        self.M = self.__check_shape(M, dim=(m,m), varname="M")
        
        self.q = self.__check_shape(q, dim=(n,), varname="q") if q is not None else np.zeros(n)
        self.r = self.__check_shape(r, dim=(n,), varname="r") if r is not None else np.zeros(n)
        
        # init solutions
        self.x = np.zeros(n)
        self.y = np.zeros(m)
        self.z = np.zeros(n)
        
        # init deltas
        self.dx = np.zeros(n)
        self.dy = np.zeros(m)
        self.dz = np.zeros(n)
        
        # init B, N (temporary)
        self.B = np.full(n, True)
        self.N = np.full(n, False)
        
    def set_initial_solution(self, x, y, z):
        self.x[:] = self.__check_shape(x, dim=(n,), varname="x")
        self.y[:] = self.__check_shape(y, dim=(m,), varname="y")
        self.z[:] = self.__check_shape(z, dim=(n,), varname="z")
        
    def set_inital_active_set(self, B, N):
        assert (np.sum((B ^ N)) == self.c.shape[0]), "Sets are not valid. |Union| should be n and |Intersection| should be 0"
        self.B[:] = self.__check_shape(B, dim=(n,), varname="B")
        self.N[:] = self.__check_shape(N, dim=(n,), varname="N")
    
    def get_error(self):
        constraint_AMb_error = np.linalg.norm(self.A.dot(self.x) + self.M.dot(self.y) - self.b)
        negative_mask = self.x < 0
        constraint_x_error = np.linalg.norm(self.x[negative_mask]) if sum(negative_mask) > 0 else 0
        return constraint_AMb_error, constraint_x_error
    
    def get_solution(self):
        constraint_AMb = self.A.dot(self.x) + self.M.dot(self.y) - self.b
        assert np.allclose(constraint_AMb, 0, rtol=self.tol), constraint_AMb
        constraint_x = (self.x >= 0)
        assert np.allclose(constraint_x, True, rtol=self.tol), constraint_x
        return self.c.dot(self.x)+0.5*(self.x.dot(self.H).dot(self.x))+0.5*(self.y.dot(self.M).dot(self.y))
    
    def test_primal_feasible(self):
        condition_1 = self.A.dot(self.x) + self.M.dot(self.y) - self.b
        assert np.allclose(condition_1, 0, rtol=self.tol), condition_1
        condition_2 = (self.H[self.B,:][:,self.B].dot(self.x[self.B]) +
            self.H[self.B,:][:,self.N].dot(self.x[self.N]) + self.c[self.B] - 
            self.A[:,self.B].transpose().dot(self.y) - self.z[self.B])
        assert np.allclose(condition_2, 0, rtol=self.tol), condition_2
        condition_3 = (self.H[self.B,:][:,self.N].transpose().dot(self.x[self.B]) + 
            self.H[self.N,:][:,self.N].dot(self.x[self.N]) + self.c[self.N] - 
            self.A[:,self.N].transpose().dot(self.y) - self.z[self.N])
        assert np.allclose(condition_3, 0, rtol=self.tol), condition_3
        condition_4 = self.z[self.B] + self.r[self.B]
        assert np.allclose(condition_4, 0, rtol=self.tol), condition_4
        condition_5 = self.x[self.N] + self.q[self.N]
        assert np.allclose(condition_5, 0, rtol=self.tol), condition_5
        condition_6 = (self.x[self.B] + self.q[self.B] >= 0)
        assert np.allclose(condition_6, True, rtol=self.tol), condition_6
        return True
    
    def test_dual_feasible(self):
        condition_1 = self.H.dot(self.x) + self.c - A.transpose().dot(self.y) - self.z
        assert np.allclose(condition_1, 0, rtol=self.tol), condition_1
        condition_2 = self.A.dot(self.x) + self.M.dot(self.y) - self.b
        assert np.allclose(condition_2, 0, rtol=self.tol), condition_2
        condition_3 = self.x[self.N] + self.q[self.N]
        assert np.allclose(condition_3, 0, rtol=self.tol), condition_3
        condition_4 = self.z[self.B] + self.r[self.B]
        assert np.allclose(condition_4, 0, rtol=self.tol), condition_4
        condition_5 = (self.z[self.N] + self.r[self.N] >= 0)
        assert np.allclose(condition_5, True, rtol=self.tol), condition_5
        return True

## Primal Active Set

In [3]:
def primal_active_set_base(self, l, debug=False):
    if debug: 
        print("===STARTING BASE ITERATION===")
        print(f"index l: {repr(l)}")
        print(f"set B: {repr(self.B)}")
        print(f"set N: {repr(self.N)}")
    # compute dx, dy, dz
    # block and B_size are temporary mockups
    # all [rows,:][:,columns].transpose() should be inverted instead
    self.dx[l] = 1
    K_B = np.block([
        [self.H[self.B,:][:,self.B], self.A[:,self.B].transpose()],
        [       self.A[:,self.B]   ,         - self.M            ]
    ])
    B_size = np.sum((self.B))
    to_solve_A = K_B
    # using H[self.N,l] here doesn't work...
    to_solve_b = - np.concatenate([self.H[self.B,:][:,l], self.A[:,l]])
    sol = np.linalg.solve(to_solve_A, to_solve_b).reshape((B_size+self.y.size,)) # numpy jank indexing
    self.dx[self.B], self.dy[:] = sol[:B_size], -sol[B_size:]
    # using H[self.N,:][:,l] here doesn't work...
    # indexing this way might have issues when B or N have 1 element
    self.dz[self.N] = (
        self.H[self.N,l]*self.dx[l] 
        + self.H[self.B,:][:,self.N].transpose().dot(self.dx[self.B])
        - self.A[:,self.N].transpose().dot(self.dy)
    )
    self.dz[l] = (
        self.H[l,l]*self.dx[l]
        + self.H[self.B,:][:,l].transpose().dot(self.dx[self.B])
        - self.A[:,l].transpose().dot(self.dy)
    )
    if debug: 
        print(f"dx: {repr(self.dx)}")
        print(f"dy: {repr(self.dy)}")
        print(f"dz: {repr(self.dz)}")
    
    # compute alpha_star
    if np.allclose(self.dz[l], 0, rtol=self.tol):    # dz[l] == 0
        alpha_star = np.inf
    else:
        alpha_star = -(self.z[l] + self.r[l]) / self.dz[l]
    if debug: print(f"alpha*: \t{str(alpha_star)}")
    
    # compute alpha_max
    neg_dx_mask = (self.dx < 0)
    to_min = self.x + self.q
    to_min[~neg_dx_mask] = np.inf
    to_min[neg_dx_mask] /= (-self.dx[neg_dx_mask])
    k = np.argmin(to_min)
    alpha_max = to_min[k]
    if debug: print(f"alpha_max: \t{str(alpha_max)}")
    if debug: print(f"k: {str(k)}")
    
    alpha = min(alpha_star, alpha_max)
    if debug: print(f"alpha: \t\t{str(alpha)}")
    
    if np.isinf(alpha):
        # terminate primal is unbounded (dual is unfeasible)
        raise Exception("Primal is Unbounded (Dual is Unfeasible)")
    
    # compute update
    self.x[l] += alpha*self.dx[l]
    self.x[self.B] += alpha*self.dx[self.B]
    self.y[:] += alpha*self.dy
    self.z[l] += alpha*self.dz[l]
    self.z[self.N] += alpha*self.dz[self.N]
    if debug: 
        print(f"x: {repr(self.x)}")
        print(f"y: {repr(self.y)}")
        print(f"z: {repr(self.z)}")
    
    if alpha == alpha_max:
        self.B[k], self.N[k] = False, True

In [4]:
def primal_active_set_intermediate(self, l, debug=False):
    if debug: 
        print("===STARTING INTERMEDIATE ITERATION===")
        print(f"index l: {repr(l)}")
        print(f"set B: {repr(self.B)}")
        print(f"set N: {repr(self.N)}")
    self.dz[l] = 1
    K_l = np.block([
        [self.H[l,l]          , self.H[self.B,:][:,l].transpose(), self.A[:,l].transpose()     ],
        [self.H[self.B,:][:,l], self.H[self.B,:][:,self.B]       , self.A[:,self.B].transpose()],
        [self.A[:,l]          , self.A[:,self.B]                 , - self.M                    ]
    ])
    B_size = np.sum((self.B))
    to_solve_A = K_l
    to_solve_b = np.zeros(1+B_size+self.y.size)
    to_solve_b[0] = 1
    sol = np.linalg.solve(to_solve_A, to_solve_b).reshape((1+B_size+self.y.size,)) # numpy janky indexing
    self.dx[l], self.dx[self.B], self.dy[:] = sol[0], sol[1:B_size+1], -sol[B_size+1:]
    self.dz[self.N] = (
        self.H[self.N,l]*self.dx[l]
        + self.H[self.B,:][:,self.N].transpose().dot(self.dx[self.B])
        - self.A[:,self.N].transpose().dot(self.dy)
    )
    if debug: 
        print(f"dx: {repr(self.dx)}")
        print(f"dy: {repr(self.dy)}")
        print(f"dz: {repr(self.dz)}")
    
    # compute alpha_star
    alpha_star = - (self.z[l] + self.r[l])
    if debug: print(f"alpha*: \t{str(alpha_star)}")
    
    # compute alpha_max
    neg_dx_mask = (self.dx < 0)
    to_min = self.x + self.q
    to_min[~neg_dx_mask] = np.inf
    to_min[neg_dx_mask] /= (-self.dx[neg_dx_mask])
    k = np.argmin(to_min)
    alpha_max = to_min[k]
    if debug: print(f"alpha_max: \t{str(alpha_max)}")
    if debug: print(f"k: {str(k)}")
    
    alpha = min(alpha_star, alpha_max)
    if debug: print(f"alpha: \t\t{str(alpha)}")
    
    if np.isclose(alpha, 0, rtol=self.tol):
        # some issues no clue why this happens...
        raise Exception("Step size is 0")
    
    # compute update
    self.x[l] += alpha*self.dx[l]
    self.x[self.B] += alpha*self.dx[self.B]
    self.y[:] += alpha*self.dy
    self.z[l] += alpha*self.dz[l]
    self.z[self.N] += alpha*self.dz[self.N]
    
    if debug: 
        print(f"x: {repr(self.x)}")
        print(f"y: {repr(self.y)}")
        print(f"z: {repr(self.z)}")
    
    if alpha == alpha_max:
        self.B[k], self.N[k] = False, True

In [5]:
def solve_primal(self, debug=False, test_invariant=False, print_error=False):
    # find x,y,z and N,B that satisfy conditions
    while True:
        l_list = np.argwhere((self.z + self.r) < 0)
        if debug: print(f"Indices that violate dual: {l_list.flatten()}")
        if l_list.size == 0:
            break
        l = l_list[0]
        self.N[l] = False
        primal_active_set_base(self, l, debug=debug)
        while (self.z[l] + self.r[l]) < 0:
            primal_active_set_intermediate(self, l, debug=debug)
        self.B[l] = True
        if print_error: print("eq. error: {}\ninq. error: {}".format(*self.get_error()))
        if test_invariant: print(f"Primal Feasibile: {self.test_primal_feasible()}")

Intial primal-feasable soluton conditions:
- $A_Bx_B + A_Nx_N + My = b$
- $H_{BB}x_B + H_{BN}x_N + c_B - A^T_By - z_B = 0$
- $H^T_{BN}x_B + H_{NN}x_N + c_N - A^T_Ny - z_N = 0$
- $x_N + q_N = 0$, 
- $z_B + r_B = 0$, 
- $x_B + q_B \geq 0$

#### Example 1: Simple Primal-Feasible Solution:
- A, M, H = eye()
- B = \[0,1\], N = \[2,3\]

In [6]:
n, m = 4, 6

# we first fix B and N
B = np.array([True, True, False, False])
N = ~B  # bitwise not

# take simple matrixes like eye
A = np.eye(m, n)
M = np.eye(m, m)
H = np.eye(n, n)

# c1: [x,0,0] + y = b
# c2: x_B + c_B - y[0,1] - z_B = 0
# c3: x_N + c_N - y[2,3] - z_N = 0
# c4: x_B >= 0, x_N = 0, z_B = 0
# base: z.N < 0

# don't ask how i got these values, i don't know it myself
b = np.array([1, 1,  2,  2,  0,  0])
c = np.array([-1, -1, 1, 1])

x = np.array([1, 1, 0, 0])
y = np.array([0, 0, 2, 2, 0, 0])
z = np.array([0, 0, -1, -1])

In [7]:
AM = np.block([A, M])
K_B = np.block([
    [H[B,:][:,B], A[:,B].T],
    [A[:,B]     , - M     ]
])
assert (np.linalg.matrix_rank(AM) == m), "(A M) has not full rank"
assert (np.linalg.matrix_rank(K_B) == m+sum(B)), "B is not second order consistent"

In [8]:
Q = QuadraticProblem(A, b, c, H, M, q=None, r=None, tol=1e-8)
Q.set_initial_solution(x,y,z)
Q.set_inital_active_set(B, N)

In [9]:
print("Primal Feasible:", Q.test_primal_feasible())
try:
    Q.test_dual_feasible()
except AssertionError as err:
    print(f"Dual Feasible: False")

Primal Feasible: True
Dual Feasible: False


In [10]:
solve_primal(Q, debug=True, test_invariant=True)

Indices that violate dual: [2 3]
===STARTING BASE ITERATION===
index l: array([2])
set B: array([ True,  True, False, False])
set N: array([False, False, False,  True])
dx: array([0., 0., 1., 0.])
dy: array([ 0.,  0., -1.,  0.,  0.,  0.])
dz: array([0., 0., 2., 0.])
alpha*: 	[0.5]
alpha_max: 	inf
k: 0
alpha: 		[0.5]
x: array([1. , 1. , 0.5, 0. ])
y: array([0. , 0. , 1.5, 2. , 0. , 0. ])
z: array([ 0.,  0.,  0., -1.])
Primal Feasibile: True
Indices that violate dual: [3]
===STARTING BASE ITERATION===
index l: array([3])
set B: array([ True,  True,  True, False])
set N: array([False, False, False, False])
dx: array([0., 0., 0., 1.])
dy: array([ 0.,  0.,  0., -1.,  0.,  0.])
dz: array([0., 0., 2., 2.])
alpha*: 	[0.5]
alpha_max: 	inf
k: 0
alpha: 		[0.5]
x: array([1. , 1. , 0.5, 0.5])
y: array([0. , 0. , 1.5, 1.5, 0. , 0. ])
z: array([0., 0., 0., 0.])
Primal Feasibile: True
Indices that violate dual: []


In [11]:
Q.get_solution()

2.5

In [12]:
Q.test_dual_feasible()

True

#### Example 2: with Intermediate Interations
I need $\Delta x_B$ to go outside bound when making the first step
to increase $\Delta x_l$ meaning some value in $x_B$ has to be linearly correlated with $x_l$. To do this we need a non identity matrix and $x_B = 0$.

In [13]:
n, m = 4, 6

# we first fix B and N
B = np.array([True, True, False, False])
N = ~B  # bitwise not

# take simple matrixes like eye
A = np.eye(m, n)
A[2,1] = 1  # add component 1 to component 2
M = np.eye(m, m)
H = np.eye(n, n)

# c1 a: for index [0,1,3,4] : [x,0,0] + y = b for 
# c1 b: for index [2]: x[1] + x[2] + y[2] = b[2]
# c2: x_B + c_B - y[0,1] - z_B = 0
# c3: x_N + c_N - y[2,3] - z_N = 0
# c4: x_B >= 0, x_N = 0, z_B = 0
# base: z.N < 0

b = np.array([1, 1,  2,  2,  0,  0])
c = np.array([-1, 3, 1, 1])

x = np.array([1, 0, 0, 0])
y = np.array([0, 1, 2, 2, 0, 0])
z = np.array([0, 0, -1, -1])

In [14]:
AM = np.block([A, M])
K_B = np.block([
    [H[B,:][:,B], A[:,B].T],
    [A[:,B]     , - M     ]
])
assert (np.linalg.matrix_rank(AM) == m), "(A M) has not full rank"
assert (np.linalg.matrix_rank(K_B) == m+sum(B)), "B is not second order consistent"

In [15]:
Q = QuadraticProblem(A, b, c, H, M, q=None, r=None, tol=1e-8)
Q.set_initial_solution(x,y,z)
Q.set_inital_active_set(B, N)

In [16]:
print("Primal Feasible:", Q.test_primal_feasible())
try:
    Q.test_dual_feasible()
except AssertionError as err:
    print(f"Dual Feasible: False")

Primal Feasible: True
Dual Feasible: False


In [17]:
try:
    solve_primal(Q, debug=True, test_invariant=True)
except Exception as e:
    print("Error:", e)

Indices that violate dual: [2 3]
===STARTING BASE ITERATION===
index l: array([2])
set B: array([ True,  True, False, False])
set N: array([False, False, False,  True])
dx: array([ 0.        , -0.33333333,  1.        ,  0.        ])
dy: array([ 0.        ,  0.33333333, -0.66666667,  0.        ,  0.        ,
        0.        ])
dz: array([0.        , 0.        , 1.66666667, 0.        ])
alpha*: 	[0.6]
alpha_max: 	0.0
k: 1
alpha: 		0.0
x: array([1., 0., 0., 0.])
y: array([0., 1., 2., 2., 0., 0.])
z: array([ 0.,  0., -1., -1.])
===STARTING INTERMEDIATE ITERATION===
index l: array([2])
set B: array([ True, False, False, False])
set N: array([False,  True, False,  True])
dx: array([ 0.        , -0.33333333,  0.5       ,  0.        ])
dy: array([ 0. ,  0. , -0.5,  0. ,  0. ,  0. ])
dz: array([0. , 0.5, 1. , 0. ])
alpha*: 	[1.]
alpha_max: 	0.0
k: 1
alpha: 		0.0
Error: Step size is 0


I have no clue as to why it is not working every pass values is valid:
- (A M) has full rank
- M, N are positive definite and symmetric
- B is second-order consistent
- the initial solution (x,y,z) is primal feasible

Maybe there are no solution (contraint generate a null set)?

## Dual Active Set

In [18]:
def dual_active_set_base(self, l, debug=False):
    if debug: 
        print("===STARTING BASE ITERATION===")
        print(f"index l: {repr(l)}")
        print(f"set B: {repr(self.B)}")
        print(f"set N: {repr(self.N)}")
    # compute dx, dy, dz
    # block and B_size are temporary mockups
    # all [rows,:][:,columns].transpose() should be inverted instead
    self.dz[l] = 1
    K_l = np.block([
        [self.H[l,l]          , self.H[self.B,:][:,l].transpose(), self.A[:,l].transpose()     ],
        [self.H[self.B,:][:,l], self.H[self.B,:][:,self.B]       , self.A[:,self.B].transpose()],
        [self.A[:,l]          , self.A[:,self.B]                 , - self.M                    ]
    ])
    B_size = np.sum((self.B))
    to_solve_A = K_l
    to_solve_b = np.zeros(1+B_size+self.y.size)
    to_solve_b[0] = 1
    sol = np.linalg.solve(to_solve_A, to_solve_b).reshape((1+B_size+self.y.size,))
    self.dx[l], self.dx[self.B], self.dy[:] = sol[0], sol[1:B_size+1], -sol[B_size+1:]
    self.dz[self.N] = (
        self.H[self.N,l]*self.dx[l] 
        + self.H[self.B,:][:,self.N].transpose().dot(self.dx[self.B])
        - self.A[:,self.N].transpose().dot(self.dy)
    )
    if debug: 
        print(f"dx: {repr(self.dx)}")
        print(f"dy: {repr(self.dy)}")
        print(f"dz: {repr(self.dz)}")
    
    # compute alpha_star
    if np.allclose(self.dx[l], 0, rtol=self.tol):    # dx[l] == 0
        alpha_star = np.inf
    else:
        alpha_star = - (self.x[l] + self.q[l]) / self.dx[l]
    if debug: print(f"alpha*: \t{str(alpha_star)}")
    
    # compute alpha_max
    neg_dz_mask = (self.dz < 0)
    to_min = self.z + self.r
    to_min[~neg_dz_mask] = np.inf
    to_min[neg_dz_mask] /= (-self.dz[neg_dz_mask])
    k = np.argmin(to_min)
    alpha_max = to_min[k]
    if debug: print(f"alpha_max: \t{str(alpha_max)}")
    if debug: print(f"k: {str(k)}")
    
    alpha = min(alpha_star, alpha_max)
    if debug: print(f"alpha: \t\t{str(alpha)}")
    
    if np.isinf(alpha):
        # terminate dual is unbounded (primal is unfeasible)
        raise Exception("Dual is Unbounded (Primal is Unfeasible)")
    
    # compute update
    self.x[l] += alpha*self.dx[l]
    self.x[self.B] += alpha*self.dx[self.B]
    self.y[:] += alpha*self.dy
    self.z[l] += alpha*self.dz[l]
    self.z[self.N] += alpha*self.dz[self.N]
    
    if debug: 
        print(f"x: {repr(self.x)}")
        print(f"y: {repr(self.y)}")
        print(f"z: {repr(self.z)}")
    
    if alpha == alpha_max:
        self.B[k], self.N[k] = True, False

In [19]:
def dual_active_set_intermediate(self, l, debug=False):
    self.dx[l] = 1
    K_B = np.block([
        [self.H[self.B,:][:,self.B], self.A[:,self.B].transpose()],
        [       self.A[:,self.B]   ,         - self.M            ]
    ])
    B_size = np.sum((self.B))
    to_solve_A = K_B
    to_solve_b = - np.concatenate([self.H[self.B,:][:,l], self.A[:,l]])
    sol = np.linalg.solve(to_solve_A, to_solve_b).reshape((B_size+self.y.size,))
    self.dx[self.B], self.dy[:] = sol[:B_size], -sol[B_size:]
    self.dz[self.N] = (
        self.H[self.N,:][:,l]*self.dx[l]
        + self.H[self.B,:][:,self.N].transpose().dot(self.dx[self.B])
        - self.A[:,self.N].transpose().dot(self.dy)
    )
    self.dz[l] = (
        self.H[l,:][:,l]*self.dx[l]
        + self.H[self.B,:][:,l].transpose().dot(self.dx[self.B])
        - self.A[:,l].transpose().dot(self.dy)
    )
    
    # compute alpha_star
    alpha_star = - (self.x[l] + self.q[l])
    
    # compute alpha_max
    neg_dz_mask = (self.dz < 0)
    to_min = self.z + self.r
    to_min[~neg_dz_mask] = np.inf
    to_min[neg_dz_mask] /= (-self.dz[neg_dz_mask])
    k = np.argmin(to_min)
    alpha_max = to_min[k]
    if debug: print(f"alpha_max: \t{str(alpha_max)}")
    if debug: print(f"k: {str(k)}")
    
    alpha = np.min(alpha_star, alpha_max)
    
    if np.isclose(alpha, 0, rtol=self.tol):
        # some issues no clue why this happens...
        raise Exception("Step size is 0")
    
    # compute update
    self.x[l] += alpha*self.dx[l]
    self.x[self.B] += alpha*self.dx[self.B]
    self.y[:] += alpha*self.dy
    self.z[l] += alpha*self.dz[l]
    self.z[self.N] += alpha*self.dz[self.N]
    
    if alpha == alpha_max:
        self.B[k], self.N[k] = True, False

In [20]:
def solve_dual(self, debug=False, test_invariant=False, print_error=False):
    # find x,y,z and N,B that satisfy conditions
    while True:
        l_list = np.argwhere((self.x + self.q) < 0)
        if debug: print(f"Indices that violate primal: {l_list.flatten()}")
        if l_list.size == 0:
            break
        l = l_list[0]
        self.B[l] = False
        dual_active_set_base(self, l, debug=debug)
        while (self.x[l] + self.q[l]) < 0:
            dual_active_set_intermediate(self, l, debug=debug)
        self.N[l] = True
        if print_error: print("eq. error: {}\ninq. error: {}".format(*self.get_error()))
        if test_invariant: print(f"Dual Feasibile: {self.test_dual_feasible()}")

Initial dual-feasable soluton conditions:
- $Hx + c - A^Ty - z = 0$
- $Ax + My - b = 0$
- $x_N + q_N = 0$, 
- $z_B + r_B = 0$, 
- $z_N + r_N \geq 0$

#### Example 1: Simple Dual-Feasible Solution: TODO
- A, M, H = eye()
- B = \[0,1\], N = \[2,3\]

In [21]:
n, m = 4, 6

# we first fix B and N
B = np.array([True, True, False, False])
N = ~B  # bitwise not

# take simple matrixes like eye
A = np.eye(m, n)
M = np.eye(m, m)
H = np.eye(n, n)

# c1: x + c - y[0,1,2,3] - z = 0
# c2: [x,0,0] + y - b = 0
# c3: z_N >= 0, z_B = 0, x_N = 0
# base: x_B < 0

b = np.array([-1, -1, 0, 0, 0, 0])
c = np.array([1, 1, 1, 1])

x = np.array([-1, -1, 0, 0])
y = np.array([0, 0, 0, 0, 0, 0])
z = np.array([0, 0, 1, 1])

In [22]:
Q = QuadraticProblem(A, b, c, H, M, q=None, r=None, tol=1e-8)
Q.set_initial_solution(x,y,z)
Q.set_inital_active_set(B, N)

In [23]:
print("Dual Feasible:", Q.test_dual_feasible())
try:
    Q.test_primal_feasible()
except AssertionError as err:
    print(f"Primal Feasible: False")

Dual Feasible: True
Primal Feasible: False


In [24]:
solve_dual(Q, debug=True, test_invariant=True)

Indices that violate primal: [0 1]
===STARTING BASE ITERATION===
index l: array([0])
set B: array([False,  True, False, False])
set N: array([False, False,  True,  True])
dx: array([0.5, 0. , 0. , 0. ])
dy: array([-0.5,  0. ,  0. ,  0. ,  0. ,  0. ])
dz: array([1., 0., 0., 0.])
alpha*: 	[2.]
alpha_max: 	inf
k: 0
alpha: 		[2.]
x: array([ 0., -1.,  0.,  0.])
y: array([-1.,  0.,  0.,  0.,  0.,  0.])
z: array([2., 0., 1., 1.])
Dual Feasibile: True
Indices that violate primal: [1]
===STARTING BASE ITERATION===
index l: array([1])
set B: array([False, False, False, False])
set N: array([ True, False,  True,  True])
dx: array([0.5, 0.5, 0. , 0. ])
dy: array([ 0. , -0.5,  0. ,  0. , -0. , -0. ])
dz: array([0., 1., 0., 0.])
alpha*: 	[2.]
alpha_max: 	inf
k: 0
alpha: 		[2.]
x: array([0., 0., 0., 0.])
y: array([-1., -1.,  0.,  0.,  0.,  0.])
z: array([2., 2., 1., 1.])
Dual Feasibile: True
Indices that violate primal: []


In [25]:
Q.get_solution()

1.0

In [26]:
Q.test_primal_feasible()

True

#### Example 2: with Intermediate Interations
I need $\alpha_{max}$ to be less then $\alpha_*$. So $\Delta z_i$ needs to be negative for some $i$.

I CANT FIGURE OUT A WAY TO DO THIS AAAHHHHH

In [27]:
n, m = 4, 6

# we first fix B and N
B = np.array([True, True, False, False])
N = ~B  # bitwise not

# take simple matrixes like eye
A = np.eye(m, n)
M = np.eye(m, m)
H = np.eye(n, n)

# if l=0 and z[2] is negative
# alpha_star = x[0]+q[0]/-(dx[0])
# alpha_max = z[2]+r[2]/-(dz[2])

# c1: x + c - y[0,1,2,3] - z = 0
# c2: [x,0,0] + y - b = 0
# c3: z_N >= 0, z_B = 0, x_N = 0
# base: x_B < 0

b = np.array([-1, 0, 0, 0, 0, 0])
c = np.array([1, 0, -1, 0])

x = np.array([-1, 0, 0, 0])
y = np.array([0, 0, 0, 0, 0, 0])
z = np.array([0, 0, -1, 0])

# r[3]>=z[3] inq. constrain
r = np.array([0, 0, 1, 0])

In [28]:
Q = QuadraticProblem(A, b, c, H, M, q=None, r=r, tol=1e-8)
Q.set_initial_solution(x,y,z)
Q.set_inital_active_set(B, N)

In [29]:
print("Dual Feasible:", Q.test_dual_feasible())
try:
    Q.test_primal_feasible()
except AssertionError as err:
    print(f"Primal Feasible: False")

Dual Feasible: True
Primal Feasible: False


In [30]:
Q.get_error()

(0.0, 1.0)

In [31]:
solve_dual(Q, debug=True, test_invariant=True, print_error=True)

Indices that violate primal: [0]
===STARTING BASE ITERATION===
index l: array([0])
set B: array([False,  True, False, False])
set N: array([False, False,  True,  True])
dx: array([0.5, 0. , 0. , 0. ])
dy: array([-0.5,  0. ,  0. ,  0. ,  0. ,  0. ])
dz: array([1., 0., 0., 0.])
alpha*: 	[2.]
alpha_max: 	inf
k: 0
alpha: 		[2.]
x: array([0., 0., 0., 0.])
y: array([-1.,  0.,  0.,  0.,  0.,  0.])
z: array([ 2.,  0., -1.,  0.])
eq. error: 0.0
inq. error: 0
Dual Feasibile: True
Indices that violate primal: []


In [32]:
Q.test_primal_feasible()

True

## ISSUES

##### Issues: Type returned is different when B or N has dimension 1

In [33]:
B = np.array([True,False,False])
N = ~B
M = np.array([[1,2,3],[4,5,6],[7,8,9]])
M[B,:][:,2]

array([3])

In [34]:
M[B,:][:,[2]]

array([[3]])

In [35]:
M[N,:][:,N]

array([[5, 6],
       [8, 9]])

## Combining Primal and Dual